# Stream active learning cycle
In this notebook, we will show how stream-based active learning strategies are used and compared them to one another. We showcase the methods available in the stream package. 

In [ ]:
import sys
sys.path.append('../..')

import numpy as np
import sklearn
import sklearn.datasets
import matplotlib.pyplot as plt
import copy
from collections import deque
from skactiveml.classifier import PWC
from skactiveml.stream import RandomSampler, PeriodicSampler
from skactiveml.stream import FixedUncertainty, VariableUncertainty, Split, PALS
from skactiveml.stream.budget_manager import FixedBudget
from skactiveml.utils import call_func

## Initialize stream Parameters
Before the experiments can start, we need to construct a random data set. For this, we specify the necessary parameters in the cell below. To make the experiment repeatable, we will use the random_state object to generate all other random seeds, such that we only need to explicitly specify a single random seed. Furthermore, we specify the length of the data stream (stream_length) and the size of the sliding window that defines the available training data (training_size).

In [ ]:
# random state that is used to generate random seeds
random_state = np.random.RandomState(0)
# number of instances that are provided to the classifier
init_train_length = 10
# the length of the data stream
stream_length = 10000
# the size of the sliding window that limits the training data
training_size = 1000

## Random seed generation
The get_randomseed function simplifies the generation of a new random seed from a given random state object. For this notebook, this random state object will be always the random_state obejct defined above.

In [ ]:
def get_randomseed(random_state):
    return random_state.randint(2**31-1)

## Generate and initialize data set
The next block initializes the tested data set. We use scikit-learn to generate a random dataset with our pre-defined stream length. The data set consists of multiple parts. X represents the location of the instance within the feature space. The class for each instance is denoted by y. 
For models that need at least some initial training data, we generate samples to train an initial model. These are denoted by the suffix "_init", while all data used within the active learning cycle are denoted by the suffix "_stream".

In [ ]:
X, y = sklearn.datasets.make_classification(n_samples=init_train_length + stream_length, random_state=get_randomseed(random_state), shuffle=True)
X_init = X[:init_train_length, :]
y_init = y[:init_train_length]
X_stream = X[init_train_length:, :]
y_stream = y[init_train_length:]

## Initialize query strategies
Next, we initialize the classifier and the base query strategies that we want to compare. To guarantee that the classifier is not affected by previous repetitions, we use factory functions to separate the classifier for each experiment run. Since all query strategies hav ea defaulr budget manag,,r we uset hat for the sake of simplicity.

In [ ]:
clf_factory = lambda: PWC(classes=[0,1], random_state=get_randomseed(random_state))
query_strategies = {
    'RandomSampler': RandomSampler(random_state=get_randomseed(random_state)),
    'PeriodicSampler': PeriodicSampler(random_state=get_randomseed(random_state)),
    'FixedUncertainty': FixedUncertainty(random_state=get_randomseed(random_state)),
    'VariableUncertainty': VariableUncertainty(random_state=get_randomseed(random_state)),
    'Split': Split(random_state=get_randomseed(random_state)),
    'PALS': PALS(random_state=get_randomseed(random_state))
}

## Start the active learning cycle
After all, variables are initialized, we can start the experiment. The experiment loop below goes through all query strategies defined in query_strategies. For each experiment run, the average accuracy of the selected query strategies will be displayed. Lastly, the accuracy over time will be plotted.

In [ ]:
for query_strategy_name, query_strategy in query_strategies.items():
    clf = clf_factory()
    # initializing the training data
    X_train = deque(maxlen=training_size)
    X_train.extend(X_init)
    y_train = deque(maxlen=training_size)
    y_train.extend(y_init)
    # train the model with the initially available data
    clf.fit(X_train, y_train)
    # initialize the list that stores the result of the classifier's prediction
    correct_classifications = []
    count = 0
    for t, (x_t, y_t) in enumerate(zip(X_stream, y_stream)):
        X_cand = x_t.reshape([1, -1])
        y_cand = y_t
        correct_classifications.append(clf.predict(X_cand)[0] == y_cand)
        # check whether to sample the instance or not
        # call_func is used since a classifier is not needed for RandomSampler and PeriodicSampler
        sampled_indices, utilities = call_func(query_strategy.query, X_cand=X_cand, clf=clf, return_utilities=True)
        # create budget_manager_param_dict for budget manager BIQF
        budget_manager_param_dict = {"utilities": utilities}
        # update the query strategy and budget_manager to calculate the right budget
        query_strategy.update(X_cand, sampled_indices, budget_manager_param_dict)
        if len(sampled_indices):
            count += 1
        X_train.append(x_t)
        y_train.append(y_cand if len(sampled_indices) > 0 else clf.missing_label)
        # train the classifier
        clf.fit(X_train, y_train)
    # calculate and show the average accuracy 
    print("Query Strategy: ", query_strategy_name, ", Avg Accuracy: ", np.sum(correct_classifications)/stream_length, ", Acquisation count:", count)
    cumsum_correct_classifications = np.cumsum(correct_classifications)
    # smoothing the accuracy for plotting
    smoothing_window_length = 100
    plt.plot((cumsum_correct_classifications[smoothing_window_length:]-cumsum_correct_classifications[:-smoothing_window_length])/smoothing_window_length, label=query_strategy_name)
plt.legend()